## Wishlist

- Ir completando as informações de cada filme comparando com o banco PG
- Adicionar atores no data scraper
- Adicionar leitor de zip

## Pegando dados do PG

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PgConnection import ReturningDF

# Define the query to fetch data
query = 'SELECT * FROM public.moviesdb'

# Call the ReturningDF function
ltbd_pg = ReturningDF(query)

# Check the result
if ltbd_pg is not None:
    print(ltbd_pg.head())
else:
    print("Failed to retrieve data.")


Empty DataFrame
Columns: [date, name, year, ltbxd_url, director, genres, themes, actors]
Index: []


## Datascraper - Adicionando os dados

In [2]:
ltbxd_scp = pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\watched.csv')

In [3]:
ltbxd_scp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            941 non-null    object
 1   Name            941 non-null    object
 2   Year            941 non-null    int64 
 3   Letterboxd URI  941 non-null    object
dtypes: int64(1), object(3)
memory usage: 29.5+ KB


In [6]:
from PgConnection import AddRowToTable

ltbxd_scp['Director'] = np.nan
ltbxd_scp['Director'] = ltbxd_scp['Director'].astype(object)

ltbxd_scp['Genres'] = np.nan
ltbxd_scp['Genres'] = ltbxd_scp['Genres'].astype(object)

# Assuming ltbxd_raw is your DataFrame
for index, row in ltbxd_scp.iterrows():
    url = row['Letterboxd URI']
    
    # Add user-agent headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    # Sending a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # Checking if the request was successful (status code 200)
    if response.status_code == 200:
        # Parsing the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Now you can work with the parsed HTML content
        # For example, you can extract specific information from the webpage
        
        # For demonstration, let's print the title of the webpage
        print(f"Title of webpage {index}: {soup.title.string}")
        # Parse the HTML

        # Find the span with class 'directorlist'
        director_span = soup.find('span', class_='directorlist')

        # Extract the director's name from the span
        director_name = director_span.text.strip()
        
        ltbxd_scp.at[index, 'Director'] = director_name
        
        # Extract genres from the specific section
        genre_section = soup.find('div', id='tab-genres')
        if genre_section:
            genre_tags = genre_section.find_all('a', class_='text-slug')
            genres = [tag.text for tag in genre_tags if '/films/genre/' in tag['href']]
        else:
            genres = ["Genres not found"]
        
        # Extract themes from the specific section
        if genre_section:
            theme_tags = genre_section.find_all('a', class_='text-slug')
            themes = [tag.text for tag in theme_tags if '/films/theme/' in tag['href'] or '/films/mini-theme/' in tag['href']]
        else:
            themes = ["Themes not found"]
        
        # Update the DataFrame with the genres and themes
        ltbxd_scp.at[index, 'Genres'] = '; '.join(genres)
        ltbxd_scp.at[index, 'Themes'] = '; '.join(themes)

        
        ltbxd_scp['Date'] = pd.to_datetime(ltbxd_scp['Date']) 
        AddRowToTable('public.moviesdb', ltbxd_scp.iloc[index])
    else:
        print(f"Failed to fetch data from URL {url} with status code {response.status_code}")

Title of webpage 0: ‎Barbie (2023) directed by Greta Gerwig • Reviews, film + cast • Letterboxd


TypeError: Connection.execute() got an unexpected keyword argument 'Date'

In [24]:
#adding ratings
ratings_raw =  pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\ratings.csv')

ltbxd_wtchd = ltbxd_scp.merge(ratings_raw[['Name','Rating']],how='left',on='Name')

## PG connection - Adicionando dados no postgres